# Morse keying recognition using 1D Convolutional Neural Network

In this playbook we will try 1D CNN in place of the LSTM RNN to perform Morse keying features recognition (dits, dahs and various silences). The first cells to prepare the data are similar to those already developed for LSTM

## Create string

Each character in the alphabet should happen a large enough number of times. As a rule of thumb we will take some multiple of the number of characters in the alphabet. If the multiplier is large enough the probability of each character appearance will be even over the alphabet. 

In [ ]:
import MorseGen

morse_gen = MorseGen.Morse()
alphabet = morse_gen.alphabet36
print(132/len(alphabet))

morsestr = MorseGen.get_morse_str(nchars=132*2, nwords=27*2, chars=alphabet)
print(alphabet)
print(len(morsestr), morsestr)

## Generate dataframe and extract envelope

In [ ]:
Fs = 8000
samples_per_dit = morse_gen.nb_samples_per_dit(Fs, 13)
n_prev = int((samples_per_dit/128)*19) + 1
print(f'Samples per dit at {Fs} Hz is {samples_per_dit}. Decimation is {samples_per_dit/128:.2f}. Look back is {n_prev}.')
label_df = morse_gen.encode_df_decim_str(morsestr, samples_per_dit, 128, alphabet)
env = label_df['env'].to_numpy()
print(type(env), len(env))

In [ ]:
import numpy as np

def get_new_data(morse_gen, SNR_dB=-23, nchars=132, nwords=27, phrase=None, alphabet="ABC"):
    if not phrase:
        phrase = MorseGen.get_morse_str(nchars=nchars, nwords=nwords, chars=alphabet)
    print(len(phrase), phrase)
    Fs = 8000
    samples_per_dit = morse_gen.nb_samples_per_dit(Fs, 13)
    n_prev = int((samples_per_dit/128)*19) + 1 # number of samples to look back is slightly more than a dit-dah and a word space (2+3+7=12)
    print(f'Samples per dit at {Fs} Hz is {samples_per_dit}. Decimation is {samples_per_dit/128:.2f}. Look back is {n_prev}.')
    label_df = morse_gen.encode_df_decim_str(phrase, samples_per_dit, 128, alphabet)
    # extract the envelope
    envelope = label_df['env'].to_numpy()
    # remove the envelope
    label_df.drop(columns=['env'], inplace=True)
    SNR_linear = 10.0**(SNR_dB/10.0)
    SNR_linear *= 256 # Apply original FFT
    print(f'Resulting SNR for original {SNR_dB} dB is {(10.0 * np.log10(SNR_linear)):.2f} dB')
    t = np.linspace(0, len(envelope)-1, len(envelope))
    power = np.sum(envelope**2)/len(envelope)
    noise_power = power/SNR_linear
    noise = np.sqrt(noise_power)*np.random.normal(0, 1, len(envelope))
    # noise = butter_lowpass_filter(raw_noise, 0.9, 3) # Noise is also filtered in the original setup from audio. This empirically simulates it
    signal = (envelope + noise)**2
    signal[signal > 1.0] = 1.0 # a bit crap ...
    return envelope, signal, label_df, n_prev

Try it...

In [ ]:
import matplotlib.pyplot as plt 

envelope, signal, label_df, n_prev = get_new_data(morse_gen, SNR_dB=-17, phrase=morsestr, alphabet=alphabet)

# Show
print(n_prev)
print(type(signal), signal.shape)
print(type(label_df), label_df.shape)
print(max(signal))
    
x0 = 0
x1 = 1500

plt.figure(figsize=(50,6))
plt.plot(signal[x0:x1]*0.9, label="sig")
plt.plot(envelope[x0:x1]*0.9, label='env')
plt.plot(label_df[x0:x1].dit*0.9 + 1.0, label='dit')
plt.plot(label_df[x0:x1].dah*0.9 + 1.0, label='dah')
plt.plot(label_df[x0:x1].ele*0.9 + 2.0, label='ele')
plt.plot(label_df[x0:x1].chr*0.9 + 2.0, label='chr')
plt.plot(label_df[x0:x1].wrd*0.9 + 2.0, label='wrd')
plt.title("signal and keying labels")
plt.legend()
plt.grid()

## Create data loader for keying model
### Define keying dataset

In [ ]:
import torch

class MorsekeyingDataset(torch.utils.data.Dataset):
    def __init__(self, morse_gen, device, SNR_dB=-23, nchars=132, nwords=27, phrase=None, alphabet="ABC"):
        self.envelope, self.signal, self.label_df0, self.seq_len = get_new_data(morse_gen, SNR_dB=SNR_dB, phrase=phrase, alphabet=alphabet)
        self.label_df = self.label_df0[['dit','dah','ele','chr','wrd']]
        self.X = torch.FloatTensor(self.signal).to(device)
        self.y = torch.FloatTensor(self.label_df.values).to(device)
        
    def __len__(self):
        return self.X.__len__() - self.seq_len

    def __getitem__(self, index):
        return (self.X[index:index+self.seq_len], self.y[index+self.seq_len])
    
    def get_envelope(self):
        return self.envelope
    
    def get_signal(self):
        return self.signal
    
    def get_labels(self):
        return self.label_df
    
    def get_labels0(self):
        return self.label_df0
    
    def get_seq_len(self):
        return self.seq_len()

### Define keying data loader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_keying_dataset = MorsekeyingDataset(morse_gen, device, -20, 132*5, 27*5, morsestr, alphabet)
train_keying_loader = torch.utils.data.DataLoader(train_keying_dataset, batch_size=1, shuffle=False) # Batch size must be 1

In [ ]:
signal = train_keying_dataset.get_signal()
signal = (signal - min(signal))
label_df = train_keying_dataset.get_labels()
label_df0 = train_keying_dataset.get_labels0()

print(type(signal), signal.shape)
print(type(label_df), label_df.shape)

x0 = 0
x1 = 1500

plt.figure(figsize=(50,6))
plt.plot(signal[x0:x1]*0.9, label="sig")
plt.plot(envelope[x0:x1]*0.9, label='env')
plt.plot(label_df[x0:x1].dit*0.9 + 1.0, label='dit')
plt.plot(label_df[x0:x1].dah*0.9 + 1.0, label='dah')
plt.plot(label_df[x0:x1].ele*0.9 + 2.0, label='ele')
plt.plot(label_df[x0:x1].chr*0.9 + 2.0, label='chr')
plt.plot(label_df[x0:x1].wrd*0.9 + 2.0, label='wrd')
plt.title("keying - signal and labels")
plt.legend()
plt.grid()

## Create model classes

The model classes are the same they will be instantiated differently for keying and character models 

### Create model for keying recognition

In [ ]:
import torch
import torch.nn as nn

cnn = nn.Conv1d(in_channels=1, out_channels=7, kernel_size=3, stride=1)
lin = nn.Linear(7, 5)
input_1d = torch.FloatTensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
seq_len = print(len(input_1d))
input_1d = input_1d.unsqueeze(0).unsqueeze(0)
print(input_1d.shape)
out = cnn(input_1d)
print(out.shape)
out = out.reshape(1,8,7)
print(out.shape)
out = lin(out)
print(out.shape)
print(out)
# print(cnn1d__y1[-1].shape)
# print(cnn1d__y1[:,:,-1])

In [ ]:
lstm = nn.LSTM(input_size=1, hidden_size=7)

In [ ]:
import torch
import torch.nn as nn

class MorseCNN(nn.Module):
    def __init__(self, device, input_size=1, output_size=5, seq_len=70, stride=1):
        self.device = device
        self.input_size = input_size
        self.output_size = output_size
        self.cnn = torch.nn.Conv1d(in_channels=input_size, out_channels=output_size, kernel_size=seq_len)